In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
!cp "/content/drive/MyDrive/Colab Notebooks/Housing/train.csv" .
!cp "/content/drive/MyDrive/Colab Notebooks/Housing/test.csv"  .


In [46]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

def regression(x,y):

        #I include the regression models I will use.
        L=LinearRegression()
        R=Ridge()
        Lass=Lasso()
        E=ElasticNet()
        ETR=ExtraTreeRegressor()
        GBR=GradientBoostingRegressor()
        XGBC=XGBRegressor()
        dt=DecisionTreeRegressor()
        kn=KNeighborsRegressor()

        #I separate the data as train and test. (20% test)
        x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=42)

        #I put my models and model names in a list.
        algos=[L,R,Lass,E,ETR,GBR,XGBC,dt,kn]
        algo_names=['Linear','Ridge','Lasso','ElasticNet','Extra Tree','Gradient Boosting','XGradientBoosting','DecisionTree','KNeighbors']

        #I create empty lists for error rates and success rates, which I will fill in later.
        r2Score = []
        rmse = []
        mae = []

        #I am creating a dataframe as I want to see all results as a table.
        #Its columns will be 'R2_score', 'RMSE', 'MAE'. The indexes will take from the string I created for the model names.
        result=pd.DataFrame(columns=['R2_score','RMSE','MAE'],index=algo_names)

        #I run the regression algorithms for each model and find the results of r2_score, mean_absolute_error and mean_squared_error.
        #And I add these results I found to the empty lists I created above with the append () function.
        for algo in algos:
            p=algo.fit(x_train,y_train).predict(x_test)
            r2Score.append(r2_score(y_test,p))
            rmse.append(mean_squared_error(y_test,p)**.5)
            mae.append(mean_absolute_error(y_test,p))

        #I fill the columns in the table with these lists.
        result['R2_score']=r2Score
        result.RMSE=rmse
        result.MAE=mae

        #I sort my result table by r2_score value and return it.
        return result.sort_values('R2_score',ascending=False)

os.getcwd()

'/content'

In [47]:
# I import numpy/pandas for fast numerical + tabular work, matplotlib/seaborn/plotly for static & interactive plots,
# statsmodels for classical stats, scikit-learn tools (SimpleImputer, scalers, encoder) for preprocessing,
# mean_squared_error for model evaluation, VIF for multicollinearity checks, and warnings to mute clutter.
# I then read train.csv & test.csv and inspect structure and missing values.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statsmodels.api as sm
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, PowerTransformer
from sklearn.metrics import mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings("ignore")
# %matplotlib inline

train_data= pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

train_data.info()

train_data.isnull().sum()

train_data.info()

test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [58]:
train_data.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [59]:
test_data.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
5,1466,60,RL,75.0,10000,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
6,1467,20,RL,NaN,7980,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,GdPrv,Shed,500,3,2010,WD,Normal
7,1468,60,RL,63.0,8402,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal
8,1469,20,RL,85.0,10176,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2010,WD,Normal
9,1470,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,4,2010,WD,Normal


In [48]:
# I create X by removing non-predictive 'Id' and the target 'SalePrice';
# I log-transform SalePrice to stabilize variance in y.
# I then pull numeric vs. categorical feature names (numer_cols, categ_cols)
# for tailored preprocessing and display them to confirm.

X = train_data.drop(['Id', 'SalePrice'], axis=1)
y= np.log(train_data['SalePrice'])

numer_cols = X.select_dtypes(include=['int64', 'float64']).columns
categ_cols = X.select_dtypes(include=['object']).columns

numer_cols

categ_cols

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [49]:
# I build two preprocessing pipelines: numer_trans imputes missing numeric values with the mean then standard-scales them,
# while categ_trans imputes missing categorical values with the mode and one-hot encodes them (ignoring unseen categories).

from sklearn.pipeline import Pipeline
numer_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])
numer_trans

categ_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])
categ_trans

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [50]:
# I combine the numeric and categorical pipelines into a single ColumnTransformer named prepro,
# mapping numer_trans to numer_cols and categ_trans to categ_cols so the full dataset
# is automatically imputed, scaled, and one-hot encoded in one step.
from sklearn.compose import ColumnTransformer
prepro = ColumnTransformer(
    transformers=[
        ('num', numer_trans, numer_cols),
        ('cat', categ_trans, categ_cols)])
prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalf...
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object'))])

In [51]:
train_data.isnull().sum()

,0
Id,0
MSSubClass,0
MSZoning,0
LotFrontage,259
LotArea,0
...,...
MoSold,0
YrSold,0
SaleType,0
SaleCondition,0


In [52]:
# I import LogisticRegression (modeling) and train_test_split (data split), split X/y into train-test sets,
# fit the preprocessing pipeline only on X_train to avoid leakage, transform X_train, and print the
# resulting matrix shape after all imputing, scaling, and one-hot encoding.

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0)

prepro.fit(X_train)
X_train_trans = prepro.transform(X_train)
print(X_train_trans.shape)

(1095, 281)


In [53]:
# I train the model, print R² scores for train and test, optionally inspect/diagnose,
# and finally transform the competition test data (dropping ‘Id’) to generate predictions.
from sklearn.linear_model import LinearRegression


X_train_trans = prepro.transform(X_train)
X_test_trans = prepro.transform(X_test)


lin_reg = LinearRegression()
lin_reg.fit(X_train_trans, y_train)

train_score = lin_reg.score(X_train_trans, y_train)
test_score = lin_reg.score(X_test_trans, y_test)

print(f"Train score: {train_score:.2f}")
print(f"Test score: {test_score:.2f}")

train_data

regression(X_test_trans, y_test)

X_test_trans2 = prepro.transform(test_data.drop(['Id'], axis=1))

predict_test = lin_reg.predict(X_test_trans2)


Train score: 0.95
Test score: 0.66


In [54]:
# I import PolynomialFeatures to create second-degree interaction terms, enhancing the model’s capacity to fit nonlinear relationships.
# I fit/transform the training set and transform the test set with the same polynomial converter.
# I refit the existing linear-regression model on the expanded feature space and print R² scores for train and test.
# I then transform the competition test data (after preprocessing) into its polynomial form and predict house prices.

from sklearn.preprocessing import PolynomialFeatures

polynomial_converter = PolynomialFeatures(degree=2,include_bias=False)
housing_train_poly = polynomial_converter.fit_transform(X_train_trans)
housing_test_poly = polynomial_converter.transform(X_test_trans)

lin_reg.fit(housing_train_poly, y_train)

train_score = lin_reg.score(housing_train_poly, y_train)
test_score = lin_reg.score(housing_test_poly, y_test)

print(f"Train score: {train_score:.2f}")
print(f"Test score: {test_score:.2f}")

X_test_trans = prepro.transform(test_data.drop(['Id'], axis=1))
X_test_trans_poly = polynomial_converter.transform(X_test_trans)

predict_test = lin_reg.predict(X_test_trans_poly)


Train score: 1.00
Test score: 0.80


In [56]:
# I call the regression diagnostic function to evaluate how well the polynomial-features model performs on the test set
regression(housing_test_poly, y_test)

,R2_score,RMSE,MAE
Ridge,0.889129,0.130389,0.090959
Linear,0.888780,0.130593,0.091091
XGradientBoosting,0.848335,0.152501,0.115157
Gradient Boosting,0.844003,0.154663,0.103156
KNeighbors,0.773538,0.186349,0.137565
Extra Tree,0.697475,0.215382,0.157073
DecisionTree,0.646392,0.232858,0.173858
Lasso,-0.001522,0.391886,0.313910
ElasticNet,-0.002094,0.391998,0.313946


#Conclusion:
Ridge regression wins: it delivers the highest R² and the lowest RMSE/MAE, meaning it explains the most variance with the smallest average and squared errors.


